In [2]:
from bs4 import BeautifulSoup, Tag
import pandas as pd
import numpy as np
import datetime as dt
import os
import betterdata
import configparser
import json
import math

In [3]:
config = configparser.ConfigParser()
config.read('config.ini')
output_dir = config['output'].get('out_dir')
dir_credentials = config['database'].get('cred_dir')
file_credentials = config['database'].get('cred_file')

In [17]:
html = open('admin-ajax.html',encoding="utf8").read()

In [18]:
soup = BeautifulSoup(html)

In [19]:
tienen_logo = soup.find_all("img", src=True)

In [20]:
ids = []
for i in tienen_logo:
    ids.append(int(i.get('src')[-7:-4].lstrip('/')))


In [4]:

initial_date = '2022-01-01'
quarter_start = dt.datetime.strptime(initial_date, '%Y-%m-%d')
end_date = '2022-03-31'
quarter_end = dt.datetime.strptime(end_date, '%Y-%m-%d')
report_date = dt.datetime.today()
today_str = dt.date(report_date.year, report_date.month,report_date.day)
config = configparser.ConfigParser()
config.read('config.ini')
current_dir = config['output'].get('out_dir')
dir_credentials = config['database'].get('cred_dir')
name_file = config['database'].get('cred_file')


connection = betterdata.create_database_connection(betterdata.read_credentials_file_local(dir_credentials = dir_credentials, file_credentials = name_file))

In [23]:
with open('querys_redshift/new_logos.sql', 'r') as file:
    _q_logos = file.read()
    _q_logos = _q_logos.format(date_end_quarter=end_date, date_start_quarter=initial_date)
empresas_total = betterdata.get_data_sql(connection=connection, query=_q_logos, close=False)
empresas_total['primer_onb'] = empresas_total['primer_onb'].apply(lambda a: pd.to_datetime(a).date()) 

In [26]:
with open('querys_redshift/dados_baja.sql', 'r') as file:
    _q_baja = file.read()
df_baja = betterdata.get_data_sql(connection=connection, query=_q_baja, close=False)

In [28]:
empresas_total[empresas_total['id_org']==265]

,id_org,organization,inicio_cont,primer_onb,activos,total,donaciones


In [27]:
empresas_total = empresas_total[~empresas_total['id_org'].isin(df_baja['id_org'])]

In [29]:
no_logo = empresas_total[~empresas_total['id_org'].isin(ids)]
no_logo=no_logo.fillna(0)
no_logo[no_logo['donaciones']>0].to_excel('Logos_Faltantes.xlsx')

In [61]:
empresas_q4 = pd.read_excel('empresas q4.xlsx')  
empresas_q4 = empresas_q4.drop_duplicates(subset='id_org')

In [66]:
q_admins = pd.merge(empresas_total,empresas_q4, how='left',on='id_org')

In [68]:
q_admins.to_excel('Q1 2022.xlsx')

In [69]:
empresas_total

,id_org,organization,inicio_cont,primer_onb,activos,total,donaciones
0,15,Betterfly,2020-08-01,2020-08-17,444,540,4343.0
1,17,STATKRAFT,2020-11-12,2020-11-12,126,144,246.0
2,18,Demo Organization,2022-01-28,2022-01-28,2,2,1.0
3,20,YANEKEN,2020-10-26,2020-10-26,1178,1601,3988.0
4,21,Asesorias UNDA,2020-09-01,2020-09-01,56,59,458.0
...,...,...,...,...,...,...,...
927,2476,Fundación Mustakis,2022-03-03,2022-03-03,41,42,27.0
928,2477,SmartJob,2022-03-03,2022-03-03,70,71,36.0
929,2509,Balneario Algarrobo,2022-03-03,2022-03-03,3,3,NaN
930,2542,Panesco y TPatagonia,2022-03-03,2022-03-03,23,23,14.0
